In [1]:
# run once
!pip install -q langchain langchain-community langchain-text-splitters langchain-openai \
chromadb sentence-transformers pypdf python-docx datasets huggingface_hub \
transformers accelerate torch streamlit pyngrok ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.6 MB/s et

In [20]:
# 1. Install required libraries
!pip install -q langchain langchain-community langchain-huggingface chromadb sentence-transformers datasets transformers accelerate bitsandbytes tqdm

# 2. Imports
from datasets import load_dataset
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
import itertools
import tqdm

# ---------------- CONFIG ----------------
PERSIST_DIR = "/content/chroma_db"
DATASET_NAME = "wikimedia/wikipedia"
DATASET_CONFIG = "20231101.en"
SAMPLE_LIMIT = 1000
BATCH_SIZE = 100
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100
MAX_CHROMA_BATCH = 5000

# 3. Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Chroma DB
db = Chroma(persist_directory=PERSIST_DIR, embedding_function=embeddings)

# 5. Splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

# 6. Load dataset (streaming)
stream = load_dataset(DATASET_NAME, DATASET_CONFIG, split="train", streaming=True)

batch_docs = []
count = 0
batch_num = 0

# ---------------- LOAD & STORE ----------------
for example in tqdm.tqdm(itertools.islice(stream, SAMPLE_LIMIT), total=SAMPLE_LIMIT):
    text = example.get("text", "") or ""
    title = example.get("title", "") or ""
    source = example.get("url", f"{DATASET_NAME}:{example.get('id','-')}")
    full_text = (title + "\n\n" + text).strip()
    if not full_text:
        continue

    doc = Document(page_content=full_text, metadata={"source": source, "title": title, "id": example.get("id")})
    batch_docs.append(doc)
    count += 1

    if len(batch_docs) >= BATCH_SIZE:
        chunks = splitter.split_documents(batch_docs)
        for i in range(0, len(chunks), MAX_CHROMA_BATCH):
            small_batch = chunks[i:i+MAX_CHROMA_BATCH]
            db.add_documents(small_batch)
        db.persist()
        batch_num += 1
        print(f"✅ Persisted batch {batch_num} — total articles processed: {count}")
        batch_docs = []

if batch_docs:
    chunks = splitter.split_documents(batch_docs)
    for i in range(0, len(chunks), MAX_CHROMA_BATCH):
        small_batch = chunks[i:i+MAX_CHROMA_BATCH]
        db.add_documents(small_batch)
    db.persist()
    print(f"✅ Final persisted. Total articles processed: {count}")

# ---------------- FREE LLM (Hugging Face Pipeline) ----------------
# 7. Load a free model from Hugging Face
model_name = "tiiuae/falcon-7b-instruct"  # or "mistralai/Mistral-7B-Instruct-v0.2"
hf_pipeline = pipeline(
    "text-generation",
    model=model_name,
    torch_dtype="auto",
    device_map="auto",
    max_new_tokens=512,
    temperature=0.3
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 8. Build RAG Chain
retriever = db.as_retriever(search_kwargs={"k": 3})
chat_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

print("✅ Chatbot ready! Type 'exit' to quit.\n")
while True:
    q = input("You: ").strip()
    if q.lower() in ("exit", "quit"):
        break
    out = chat_chain.invoke({"query": q})
    print("\nBot:\n", out.get("result", "No answer"))
    sdocs = out.get("source_documents", [])
    if sdocs:
        print("\nSources:")
        for i, sd in enumerate(sdocs, 1):
            print(i, sd.metadata.get("source", "-"), "| title:", sd.metadata.get("title", "-"))
    print("\n---\n")


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

 10%|█         | 100/1000 [00:25<03:47,  3.96it/s]

✅ Persisted batch 1 — total articles processed: 100


 20%|██        | 200/1000 [00:42<02:41,  4.94it/s]

✅ Persisted batch 2 — total articles processed: 200


...

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Chatbot ready! Type 'exit' to quit.

You: what is the capital of Pakistan?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Bot:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Autonomous Region of China; and the Gilgit-Baltistan territory, Khyber Pakhtunkhwa province and Balochistan province of Pakistan.

Autonomous Region of China; and the Gilgit-Baltistan territory, Khyber Pakhtunkhwa province and Balochistan province of Pakistan.

Afghanistan

Afghanistan, officially the Islamic Emirate of Afghanistan, is a landlocked country located at the crossroads of Central Asia and South Asia. Referred to as the Heart of Asia, it is bordered by Pakistan to the east and south, Iran to the west, Turkmenistan to the northwest, Uzbekistan to the north, Tajikistan to the northeast, and China to the northeast and east. Occupying  of land, the country is predominantly mountainous with plains in the north and the southwest, which are separated by the Hindu Kush mountain range. Kabul is the country's larges